# Supervised Finetunine of Falcon 7B model on Norewegian Dialogue Dataset

In [ ]:
## Inference
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0"

from dataclasses import dataclass, field
from typing import Optional

import torch
from datasets import load_dataset
from peft import LoraConfig
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    AutoTokenizer,
    TrainingArguments,
)

from trl import SFTTrainer

from peft import (
    prepare_model_for_kbit_training,
    LoraConfig,
    get_peft_model,
    PeftModel
)

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype="bfloat16",
    bnb_4bit_use_double_quant=False,
)
device_map = {"": 0}


### Load Guanaco Instruction Dataset


In [ ]:
from datasets import load_dataset

dataset_name = "timdettmers/openassistant-guanaco"
dataset = load_dataset(dataset_name, split="train")

### Custom dataset - Norwegian Dialogue Generation Disclaimer this dataset was generated using GPT4 and should only be used for non commercial purposes.

In [ ]:

# Load the transformed data into a HuggingFace Dataset
dataset = load_dataset('json', data_files='transformed.json', split='train')
validation_dataset = load_dataset('json', data_files='transformed_valdiation.json', split='train')
# Let's check the first element in the dataset
print(dataset[0])
print(validation_dataset[0])

In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, AutoTokenizer

model_name = "tiiuae/falcon-7b"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
)

model = AutoModelForCausalLM.from_pretrained(
    model_name, 
    quantization_config=bnb_config, 
    trust_remote_code=True
)
model.config.use_cache = False

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token

In [ ]:
## Finetune

from peft import LoraConfig

lora_alpha = 16
lora_dropout = 0.1
lora_r = 64

peft_config = LoraConfig(
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    r=lora_r,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=[
        "query_key_value",
        "dense",
        "dense_h_to_4h",
        "dense_4h_to_h",
    ]
)



In [ ]:
from transformers import TrainingArguments

output_dir = "./results"
per_device_train_batch_size = 4
gradient_accumulation_steps = 4
optim = "paged_adamw_32bit"
save_steps = 10
logging_steps = 10
learning_rate = 2e-4
max_grad_norm = 0.3
max_steps = 500
warmup_ratio = 0.03
lr_scheduler_type = "constant"
eval_steps = 10
do_eval = True
evaluation_strategy = "steps"

training_arguments = TrainingArguments(
    output_dir=output_dir,
    per_device_train_batch_size=per_device_train_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    optim=optim,
    save_steps=save_steps,
    logging_steps=logging_steps,
    learning_rate=learning_rate,
    fp16=True,
    max_grad_norm=max_grad_norm,
    max_steps=max_steps,
    warmup_ratio=warmup_ratio,
    group_by_length=True,
    lr_scheduler_type=lr_scheduler_type,
    eval_steps=eval_steps,
    do_eval=do_eval,
    evaluation_strategy=evaluation_strategy,
)

In [ ]:
from trl import SFTTrainer

max_seq_length = 512

trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    eval_dataset=validation_dataset,
    peft_config=peft_config,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    tokenizer=tokenizer,
    args=training_arguments,
)

In [ ]:
for name, module in trainer.model.named_modules():
    if "norm" in name:
        module = module.to(torch.float32)

### Finetune Model - Beware! If your dataset is small it will overfit super quickly!

In [ ]:

trainer.train()


## Load vanilla model and run inference for crude comparison

In [ ]:

tokenizer = AutoTokenizer.from_pretrained("tiiuae/falcon-7b")
tokenizer.pad_token = tokenizer.eos_token
model = AutoModelForCausalLM.from_pretrained(
    "tiiuae/falcon-7b", quantization_config=bnb_config, device_map=device_map, trust_remote_code=True,
    load_in_4bit=True
)


In [ ]:
%%timeit -r 5
### Inference

# inference test
with torch.inference_mode():
    text = '#### Human: Write a short dialogue between two Norwegian people, Cristiana and Waleed, who just met. Keep it colloquial.### Assitant: Hei, Cristiana! Hvordan går det med deg? Det er lenge siden sist vi så hverandre.\n\nCristiana: Hei, Waleed! Jo, det er virkelig for lenge siden. Tja, livet har vært ganske hektisk, men bra igjen. Hvordan står det til med deg?\n\nWaleed: Åh, du sier noe der! Jeg har så mye å gjøre med jobbintervjuer og sånt, jeg har jo ikke fått en jobb ennå.\n\nCristiana:'
    outputs = model.generate(input_ids=tokenizer(text, return_tensors="pt").input_ids.to(model.device), 
                            max_new_tokens=30, 
                            temperature=0.5, 
                            num_return_sequences=1,
                            do_sample=True,
                            use_cache = True)

print(tokenizer.batch_decode(outputs))



## Load finetuned model and run inference for crude comparison

In [ ]:

# Load finetuned model 

tokenizer = AutoTokenizer.from_pretrained("tiiuae/falcon-7b")
tokenizer.pad_token = tokenizer.eos_token
model = AutoModelForCausalLM.from_pretrained(
    "tiiuae/falcon-7b", quantization_config=bnb_config, device_map=device_map, trust_remote_code=True,
    load_in_4bit=True
)

model = prepare_model_for_kbit_training(model, use_gradient_checkpointing=False)
# change this path to your checkpoint folder path
model_id = "/home/lars/ML/LLM/results/checkpoint-30"
model = PeftModel.from_pretrained(model, model_id)
model.config.use_cache = False




In [ ]:
%%timeit -r 5
### Inference

# inference test
with torch.inference_mode():
    text = '#### Human: Write a short dialogue between two Norwegian people, Cristiana and Waleed, who just met. Keep it colloquial.### Assitant: Hei, Cristiana! Hvordan går det med deg? Det er lenge siden sist vi så hverandre.\n\nCristiana: Hei, Waleed! Jo, det er virkelig for lenge siden. Tja, livet har vært ganske hektisk, men bra igjen. Hvordan står det til med deg?\n\nWaleed: Åh, du sier noe der! Jeg har så mye å gjøre med jobbintervjuer og sånt, jeg har jo ikke fått en jobb ennå.\n\nCristiana:'
    outputs = model.generate(input_ids=tokenizer(text, return_tensors="pt").input_ids.to(model.device), 
                            max_new_tokens=30, 
                            temperature=0.5, 
                            num_return_sequences=1,
                            do_sample=True,
                            use_cache = True)

print(tokenizer.batch_decode(outputs))

